## Initialization

### Import Libraries

In [1]:
import requests
import io
import time
import calendar

In [2]:
import pandas as pd
import numpy as np
import pandas_gbq

### Default constants

In [3]:
BUCKET_NAME = "homework_rl3154_data"
PROJECT_BUCKET = "project"
FOLDER_NAME = "electricity_load_data"
BIG_QUERY_TABLE_NAME = "project_dataset.base_electricity_load_data"


# Scraping Data

This part is used to scrape electricity load data from [NYISO](https://www.nyiso.com/load-data) and insert it into Google Cloud Storage

In [4]:
# Request URL
nyiso_url = 'http://dss.nyiso.com/dss_oasis/PublicReports'

# Request headers

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:94.0) Gecko/20100101 Firefox/94.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,id;q=0.7,en;q=0.3',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Origin': 'null',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

In [ ]:
# Iterating 2010-2021
for year in range(2010,2022):
    
    # Iterating Jan - Dec
    for month in range(1,13):
        
        # get last day of the month
        last_date = calendar.monthrange(year, month)[1]
        
        # generate data format for requesting report
        data = [
          ('reportKey', 'RT_ACT_LOAD'),
          ('startDate', f'{month}/01/{year}'),
          ('endDate', f'{month}/{last_date}/{year}'),
          ('version', 'L'),
          ('dataFormat', 'CSV'),
          ('filter', 'CAPITL'),
          ('filter', 'CENTRL'),
          ('filter', 'DUNWOD'),
          ('filter', 'GENESE'),
          ('filter', 'H Q'),
          ('filter', 'HUD VL'),
          ('filter', 'LONGIL'),
          ('filter', 'MHK VL'),
          ('filter', 'MILLWD'),
          ('filter', 'N.Y.C.'),
          ('filter', 'NORTH'),
          ('filter', 'NPX'),
          ('filter', 'O H'),
          ('filter', 'PJM'),
          ('filter', 'WEST'),
        ]
        
        # Request the CSV data
        response = requests.post(nyiso_url, headers=headers, data=data)
        url_content = response.content
        temp_df = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
        
        # save to GCS Bucket
        temp_df.to_csv(f"gs://{BUCKET_NAME}/{PROJECT_BUCKET}/{FOLDER_NAME}/{year}-{month:02d}.csv", index=False)
        print(f"{year}-{month:02d} done")
        
        # finish if already Oct 2021
        if ((year==2021) and (month==10)):
            break
        
        # Add sleep so that we are not blocked from the website
        time.sleep(10)
        


2010-01 done
2010-02 done
2010-03 done
2010-04 done
2010-05 done
2010-06 done
2010-07 done
2010-08 done
2010-09 done
2010-10 done
2010-11 done
2010-12 done
2011-01 done
2011-02 done
2011-03 done
2011-04 done
2011-05 done
2011-06 done
2011-07 done
2011-08 done
2011-09 done
2011-10 done
2011-11 done
2011-12 done
2012-01 done
2012-02 done
2012-03 done
2012-04 done
2012-05 done
2012-06 done
2012-07 done
2012-08 done
2012-09 done
2012-10 done
2012-11 done
2012-12 done
2013-01 done
2013-02 done
2013-03 done
2013-04 done
2013-05 done
2013-06 done
2013-07 done
2013-08 done
2013-09 done
2013-10 done
2013-11 done
2013-12 done
2014-01 done
2014-02 done
2014-03 done
2014-04 done
2014-05 done
2014-06 done
2014-07 done
2014-08 done
2014-09 done
2014-10 done
2014-11 done
2014-12 done
2015-01 done
2015-02 done
2015-03 done
2015-04 done
2015-05 done
2015-06 done
2015-07 done
2015-08 done
2015-09 done
2015-10 done
2015-11 done
2015-12 done
2016-01 done
2016-02 done
2016-03 done
2016-04 done
2016-05 done

# Getting all base data to Google Big Query

After getting multiple files of hourly electricity data per month, we convert these data from Google Cloud Storage to a Google BigQuery database

In [4]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

In [5]:
# Iterate all the files in Google Cloud Storage
for blob in bucket.list_blobs(prefix=f"{PROJECT_BUCKET}/{FOLDER_NAME}"):
    # Read CSV
    temp_df = pd.read_csv(f"gs://{BUCKET_NAME}/{blob.name}")
    
    # Format the CSV text file into a format we want our database to be
    temp_df.columns = ['time_stamp', 'zone_name', 'zone_ptid', 'rtd_actual_load']
    temp_df['time_stamp'] = pd.to_datetime(temp_df['time_stamp'])
    
    print(blob.name.split('/')[-1])
    # Insert into Big Query
    temp_df.to_gbq(BIG_QUERY_TABLE_NAME, if_exists='append')

2010-01.csv


100%|██████████| 1/1 [00:00<00:00, 8848.74it/s]


2010-02.csv


100%|██████████| 1/1 [00:00<00:00, 12945.38it/s]


2010-03.csv


...

100%|██████████| 1/1 [00:00<00:00, 13751.82it/s]


2021-09.csv


100%|██████████| 1/1 [00:00<00:00, 11586.48it/s]


2021-10.csv


100%|██████████| 1/1 [00:00<00:00, 9137.92it/s]


# Creating table for aggregation

## Aggregate data hourly

In [4]:
sql_query = f"""
    SELECT
        zone_name
        , FORMAT_DATE('%Y-%m-%d %H:00:00', time_stamp) AS hour_timestamp
        , SUM(rtd_actual_load) AS sum_rtd_actual_load
    FROM {BIG_QUERY_TABLE_NAME}
    GROUP BY 
        1,2
"""

In [5]:
temp_df = pd.read_gbq(sql_query)
temp_df['hour_timestamp'] = pd.to_datetime(temp_df['hour_timestamp'])

In [18]:
temp_df.to_gbq("project_dataset.agg_electricity_load_data_hourly")

100%|██████████| 1/1 [00:00<00:00, 13842.59it/s]


## Aggregate data daily

In [4]:
sql_query = f"""
    SELECT
        zone_name
        , FORMAT_DATE('%Y-%m-%d', time_stamp) AS daily_timestamp
        , SUM(rtd_actual_load) AS sum_rtd_actual_load
    FROM {BIG_QUERY_TABLE_NAME}
    GROUP BY 
        1,2
"""

In [5]:
temp_df = pd.read_gbq(sql_query)
temp_df['daily_timestamp'] = pd.to_datetime(temp_df['daily_timestamp'])

In [9]:
temp_df.to_gbq("project_dataset.agg_electricity_load_data_daily")

100%|██████████| 1/1 [00:00<00:00, 10538.45it/s]
